Summarize the Results of the 07_compare_models program.

1. Load the Excel file for each model from the prior module (where each model was compared on common feature sets)
2. Match each row using a clean, sorted feature set key
3. Merge all metrics into a single, unified table: one row for each unique set of dataset features
4. Log any duplicate feature sets to a separate .txt file (there should not be any of these)
5. Compute six summary accuracy stats for each row (e.g., mean, median, best model, worst model)
6. Compute six summary runtime stats for each row (e.g., mean, meadian, fastest model, slowest model)
7. Export everything to a single Excel file.


In [ ]:
# -------------------------------------------
# 1. CONSOLIDATE MODEL COMPARISON RESULTS
# -------------------------------------------


import os
import pandas as pd
from collections import defaultdict
import numpy as np
from tqdm import tqdm

# Import config paths
import sys
sys.path.append('..')
from config import RESULTS_DIR, SUMMARY_DIR

# Define path to all ML models' Excel files, and name the output files created in this module.
# To create the file for the baseline models, add "baseline_" as a prefix to the model name, below.
# Else do not add "baseline_" to the model name to create the file for the benchmarked (after tuning hyperparameters) models.
input_dir = RESULTS_DIR / "features_14_16"
output_file = SUMMARY_DIR / "baseline_consolidated_results_features_14_16_3_loops_each_feature_set.xlsx"
duplicate_log = SUMMARY_DIR / "baseline_duplicates_log.txt"

# Ensure output directory exists
SUMMARY_DIR.mkdir(parents=True, exist_ok=True)

# Take the first 16 columns of each file (the features), remove missing or null
# values (e.g., for 14 features, there will be two columns that are blank),
# sort the remaning feature names alpha, then convet to a tuple for use as a dictionary key.
# Why?  Different Excel files will have the same feature sets, but may be in a different order.
# Not doing this would result in additional Excel rows with the same set of features. Sweet, right?
def extract_sorted_key(row):
    features = [f for f in row[:16] if pd.notna(f) and str(f).strip()]
    return tuple(sorted(features))

# Set up two data structures to collect data from each model's Excel file,and organize it for merging:
#   1. master_data will store performance metrics for each model-feature combination
#   2. feature_rows will store the actual feature names for each unique combination (e.g., "A", "P", . . . ).
# Why? Prevents the same feature set from appearing more than once for one model.
master_data = defaultdict(dict)
feature_rows = {}

# Scan all Excel files
# In filename.startswith, insert "baseline_" as prefix, below, to collect all the baseline models Excel files. 
# Else, insert "benchmark_" to collect all the benchmark models Excel files.
for filename in os.listdir(input_dir):
    if filename.endswith(".xlsx") and filename.startswith("baseline_"):
        model_name = filename.split("_")[1]  # Extract model name from benchmark_modelname_results_14_16.xlsx
        path = input_dir / filename
        df = pd.read_excel(path)

        for idx, row in df.iterrows():
            key = extract_sorted_key(row)
            if key in master_data[model_name]:
                with open(duplicate_log, "a") as f:
                    f.write(f"Duplicate detected in model: {model_name}\n")
                    f.write(f"Feature set: {key}\n\n")
                continue  # Skip duplicate

            master_data[model_name][key] = {
                f"Accuracy_{model_name}": row["Accuracy"],
                f"Runtime_{model_name}": row["Runtime (ms)"]
            }
            feature_rows.setdefault(key, list(row[:16]))

# Merge into one DataFrame
rows = []

for key in tqdm(feature_rows, desc="Merging feature sets"):
    base_row = feature_rows[key]
    row_data = {f"Feature_{i+1}": base_row[i] if i < len(base_row) else "" for i in range(16)}

    for model in master_data:
        metrics = master_data[model].get(key, {})
        row_data.update(metrics)

    rows.append((key, row_data))

df_combined = pd.DataFrame([r[1] for r in rows])


# Compute summary stats per row (per feature set), which will be columns.
accuracy_cols = [col for col in df_combined.columns if col.startswith("Accuracy_")]
runtime_cols = [col for col in df_combined.columns if col.startswith("Runtime_")]

df_combined["Mean_Accuracy"] = df_combined[accuracy_cols].mean(axis=1)
df_combined["Median_Accuracy"] = df_combined[accuracy_cols].median(axis=1)
df_combined["Max_Accuracy"] = df_combined[accuracy_cols].max(axis=1)
df_combined["Best_Accuracy_Model"] = df_combined[accuracy_cols].idxmax(axis=1).str.replace("Accuracy_", "")
df_combined["Min_Accuracy"] = df_combined[accuracy_cols].min(axis=1)
df_combined["Worst_Accuracy_Model"] = df_combined[accuracy_cols].idxmin(axis=1).str.replace("Accuracy_", "")

df_combined["Mean_Runtime (ms)"] = df_combined[runtime_cols].mean(axis=1)
df_combined["Median_Runtime (ms)"] = df_combined[runtime_cols].median(axis=1)
df_combined["Max_Runtime (ms)"] = df_combined[runtime_cols].max(axis=1)
df_combined["Slowest_Model"] = df_combined[runtime_cols].idxmax(axis=1).str.replace("Runtime_", "")
df_combined["Min_Runtime (ms)"] = df_combined[runtime_cols].min(axis=1)
df_combined["Fastest_Model"] = df_combined[runtime_cols].idxmin(axis=1).str.replace("Runtime_", "")

# Export to Excel
df_combined.to_excel(output_file, index=False)
print(f"\nSummary written to {output_file}")


Merging feature sets: 100%|██████████| 137/137 [00:00<00:00, 96965.85it/s]


✅ Summary written to C:\Misc\ml_benchmark\outputs\summary\baseline_consolidated_results_features_14_16_3_loops_each_feature_set.xlsx


In [ ]:
# -------------------------------------------
# 2. CONSOLIDATE PER-CLASS ACCURACY FROM CONFUSION MATRICES
# -------------------------------------------

import pandas as pd
from pathlib import Path

# Path to the confusion matrices Excel file (input)
conf_matrix_file = Path(RESULTS_DIR) / "confusion_matrices_all_models_14_16.xlsx"
# Path to output consolidated per-class accuracy file (no _14_16 suffix)
output_file = Path(SUMMARY_DIR) / "consolidated_per_class_accuracy_features.xlsx"

print(f"Reading confusion matrices from: {conf_matrix_file}")
print(f"Output will be saved to: {output_file}")

# Read all sheets (one per model)
all_sheets = pd.read_excel(conf_matrix_file, sheet_name=None)

consolidated = []

for model_name, df in all_sheets.items():
    # The actual confusion matrix is columns C onward (skip first two columns)
    # Reset index in case there are any issues with row labels
    matrix = df.iloc[:, 2:].apply(pd.to_numeric, errors='coerce')
    per_class_acc = {}
    for i in range(matrix.shape[0]):
        true_positives = matrix.iloc[i, i]
        total = matrix.iloc[i, :].sum()
        acc = true_positives / total if total > 0 else float('nan')
        per_class_acc[f"Accuracy_{i}"] = acc
    row = {"Model": model_name}
    row.update(per_class_acc)
    consolidated.append(row)

df_out = pd.DataFrame(consolidated)
df_out.to_excel(output_file, index=False)

print(f"\nConsolidated per-class accuracy saved to: {output_file}")
print(f"Models processed: {len(df_out)}")
print(f"Columns: {list(df_out.columns)}")
print("\nSample of consolidated data:")
print(df_out.head(3).to_string(index=False))

Reading confusion matrices from: C:\Misc\ml_benchmark\outputs\results\confusion_matrices_all_models_14_16.xlsx
Output will be saved to: C:\Misc\ml_benchmark\outputs\summary\consolidated_per_class_accuracy_features.xlsx

Consolidated per-class accuracy saved to: C:\Misc\ml_benchmark\outputs\summary\consolidated_per_class_accuracy_features.xlsx
Models processed: 16
Columns: ['Model', 'Accuracy_0', 'Accuracy_1', 'Accuracy_2', 'Accuracy_3', 'Accuracy_4', 'Accuracy_5', 'Accuracy_6']

Sample of consolidated data:
                        Model  Accuracy_0  Accuracy_1  Accuracy_2  Accuracy_3  Accuracy_4  Accuracy_5  Accuracy_6
              RidgeClassifier    0.830560    0.967433    0.940491    0.881557    0.915456    0.929946    0.900228
                   GaussianNB    0.807867    1.000000    0.899387    0.875353    0.954357    0.941786    0.871017
QuadraticDiscriminantAnalysis    0.894100    1.000000    0.952147    0.898195    0.955913    0.941786    0.880501
